<a href="https://colab.research.google.com/github/tonyyoon123/MachineLearning/blob/master/Assignment10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Import Libaries*

In [0]:
import matplotlib.pyplot as plt
import numpy as np

*Input Data*

In [0]:
file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

# normalize the values of the input data to be [0, 1]
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

# example of distance function between two vectors x and y
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

# make a matrix each column of which represents an images in a vector form
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:
  line_data   = line.split(',')
  label       = line_data[0]
  im_vector   = np.asfarray(line_data[1:])    
  list_label[count]       = label
  list_image[:, count]    = im_vector
  count += 1

#consider the first 1,000 images for training and the rest 9,000 images for testing and also transpose the vector
#train_labels = list_label[:1000]  
#train_images = list_image[:1000]  
#test_labels  = list_label[1000:]  
#test_image  = list_image[1000:] 

num_inputs = 784
num_outputs = 10
num_examples = 10000


train_imgs = np.empty((6000, size_row * size_col), dtype=float)
test_imgs = np.empty((4000, size_row * size_col), dtype=float)
train_labels = np.empty(6000, dtype=int)
test_labels = np.empty(4000, dtype=int)
for i in range(num_image):
  if i < 6000:
    train_labels[i] = list_label[i]
    train_imgs[i] = list_image[:, i]
  else:
    test_labels[i-6000] = list_label[i]
    test_imgs[i-6000] = list_image[:, i]

fac = 0.99 / 255
train_imgs = np.asfarray(train_imgs) * fac + 0.01
test_imgs = np.asfarray(test_imgs) * fac + 0.01
train_labels = np.asfarray(train_labels)
test_labels = np.asfarray(test_labels)

train_targets = np.array(train_labels).astype(np.int)
train_labels_one_hot = np.eye(np.max(train_targets) + 1)[train_targets]
test_targets = np.array(test_labels).astype(np.int)
test_labels_one_hot = np.eye(np.max(test_targets) + 1)[test_targets]

train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99


class NeuralNetwork:
  def __init__(self):
    self.lr = 0.01
    self.w1 = np.random.randn(784, 196)
    self.b1 = np.zeros((1, 196))  
    self.w2 = np.random.randn(196, 49)
    self.b2 = np.zeros((1, 49))  
    self.w3 = np.random.randn(49, 10)
    self.b3 = np.zeros((1, 10))

  def feedforward(self):
    z1 = np.dot(self.x, self.w1) + self.b1
    self.a1 = sigmoid(z1)
    z2 = np.dot(self.a1, self.w2) + self.b2
    self.a2 = sigmoid(z2)
    z3 = np.dot(self.a2, self.w3) + self.b3
    self.a3 = sigmoid(z3)      

  def compute_cost(self, y_pred, y):
    m = y.shape[1]    
    cost = np.mean(np.sum(-y*np.log(y_pred), axis=1))
    return cost

  def backprop(self):        
    output_errors = self.y - self.a3        
    a3_delta = output_errors * (self.a3 * (1.0 - self.a3)) # w3        
    z2_delta = np.dot(output_errors, self.w3.T)
    a2_delta = z2_delta * (self.a2 * (1.0 - self.a2)) # w2        
    z1_delta = np.dot(z2_delta, self.w2.T)
    a1_delta = z1_delta * (self.a1 * (1.0 - self.a1)) # w1

    self.w3 += self.lr * np.dot(self.a2.T, a3_delta)
    self.b3 += self.lr * np.sum(a3_delta, axis=0, keepdims=True)        
    self.w2 += self.lr * np.dot(self.a1.T, a2_delta)
    self.b2 += self.lr * np.sum(a2_delta, axis=0, keepdims=True)        
    self.w1 += self.lr * np.dot(self.x.T, a1_delta)
    self.b1 += self.lr * np.sum(a1_delta, axis=0, keepdims=True)    
    
  def train(self, x, y):
    self.x = np.array(x, ndmin=2)
    self.y = np.array(y, ndmin=2)
    self.feedforward()
    self.backprop()

  def predict(self, data):
    self.x = np.array(data, ndmin=2)
    self.feedforward()
    return self.a3

  def evaluate(self, x, y):
    corrects, wrongs = 0, 0
    for i in range(len(x)):
      res = self.predict(x[i])
      res_max = res.argmax()
      if res_max == y[i]:
        corrects += 1        
      else:
        wrongs += 1        
    return corrects, wrongs

model = NeuralNetwork()
epochs = 10
costs_train = []
costs_test = []
accuracys_train = []
accuracys_test = []

for epoch in range(epochs):
  print("epoch: ", epoch+1)
  for i in range(len(train_images)):
    model.train(train_images[i], train_labels_one_hot[i])  
  
  corrects, wrongs = model.evaluate(train_images, train_labels)  
  accuracys_train.append(corrects / ( corrects + wrongs))
  costs_train.append(model.compute_cost(model.predict(train_images), train_labels_one_hot))  
  corrects, wrongs = model.evaluate(test_images, test_labels)  
  accuracys_test.append(corrects / ( corrects + wrongs))
  costs_test.append(model.compute_cost(model.predict(test_images), test_labels_one_hot))

plt.plot(costs_train, c='b', label='cost train-set')
plt.plot(costs_test, c='r', label='cost test-set')
plt.xlabel("Interation")
plt.ylabel("Cost")
plt.legend()
plt.show()  

In [0]:
# Devide testing and training sets
train_imgs = np.empty((1000, size_row * size_col), dtype=float)
test_imgs = np.empty((9000, size_row * size_col), dtype=float)
train_labels = np.empty(1000, dtype=int)
test_labels = np.empty(9000, dtype=int)
for i in range(num_image):
  if i < 1000:
    train_labels[i] = list_label[i]
    train_imgs[i] = list_image[:, i]
  else:
    test_labels[i-9000] = list_label[i]
    test_imgs[i-9000] = list_image[:, i]


# Normalization
fac = 0.99 / 255
train_imgs = np.asfarray(train_imgs) * fac + 0.01
test_imgs = np.asfarray(test_imgs) * fac + 0.01
train_labels = np.asfarray(train_labels)
test_labels = np.asfarray(test_labels)

# One hot encoding our labels
train_targets = np.array(train_labels).astype(np.int)
train_labels_one_hot = np.eye(np.max(train_targets) + 1)[train_targets]
test_targets = np.array(test_labels).astype(np.int)
test_labels_one_hot = np.eye(np.max(test_targets) + 1)[test_targets]

train_labels_one_hot[train_labels_one_hot==0] = 0.01
train_labels_one_hot[train_labels_one_hot==1] = 0.99
test_labels_one_hot[test_labels_one_hot==0] = 0.01
test_labels_one_hot[test_labels_one_hot==1] = 0.99

def sigmoid(z):
  return 1 / ( 1 + np.exp(-z))

class NeuralNetwork:
  def __init__(self):
    self.lr = 0.01
    self.w1 = np.random.randn(784, 196)
    self.b1 = np.zeros((1, 196))  
    self.w2 = np.random.randn(196, 49)
    self.b2 = np.zeros((1, 49))  
    self.w3 = np.random.randn(49, 10)
    self.b3 = np.zeros((1, 10))

  def feedforward(self):
    z1 = np.dot(self.x, self.w1) + self.b1
    self.a1 = sigmoid(z1)
    z2 = np.dot(self.a1, self.w2) + self.b2
    self.a2 = sigmoid(z2)
    z3 = np.dot(self.a2, self.w3) + self.b3
    self.a3 = sigmoid(z3)      

  def compute_cost(self, y_pred, y):
    m = y.shape[1]    
    cost = np.mean(np.sum(-y*np.log(y_pred), axis=1))
    return cost

  def backprop(self):        
    output_errors = self.y - self.a3        
    a3_delta = output_errors * (self.a3 * (1.0 - self.a3)) # w3        
    z2_delta = np.dot(output_errors, self.w3.T)
    a2_delta = z2_delta * (self.a2 * (1.0 - self.a2)) # w2        
    z1_delta = np.dot(z2_delta, self.w2.T)
    a1_delta = z1_delta * (self.a1 * (1.0 - self.a1)) # w1

    self.w3 += self.lr * np.dot(self.a2.T, a3_delta)
    self.b3 += self.lr * np.sum(a3_delta, axis=0, keepdims=True)        
    self.w2 += self.lr * np.dot(self.a1.T, a2_delta)
    self.b2 += self.lr * np.sum(a2_delta, axis=0, keepdims=True)        
    self.w1 += self.lr * np.dot(self.x.T, a1_delta)
    self.b1 += self.lr * np.sum(a1_delta, axis=0, keepdims=True)    
    
  def train(self, x, y):
    self.x = np.array(x, ndmin=2)
    self.y = np.array(y, ndmin=2)
    self.feedforward()
    self.backprop()

  def predict(self, data):
    self.x = np.array(data, ndmin=2)
    self.feedforward()
    return self.a3

  def evaluate(self, x, y):
    corrects, wrongs = 0, 0
    for i in range(len(x)):
      res = self.predict(x[i])
      res_max = res.argmax()
      if res_max == y[i]:
        corrects += 1        
      else:
        wrongs += 1        
    return corrects, wrongs

model = NeuralNetwork()
epochs = 10
costs_train = []
costs_test = []
accuracys_train = []
accuracys_test = []

for epoch in range(epochs):
  print("epoch: ", epoch+1)
  for i in range(len(train_imgs)):
    model.train(train_imgs[i], train_labels_one_hot[i])  
  
  corrects, wrongs = model.evaluate(train_imgs, train_labels)  
  accuracys_train.append(corrects / ( corrects + wrongs))
  costs_train.append(model.compute_cost(model.predict(train_imgs), train_labels_one_hot))  
  corrects, wrongs = model.evaluate(test_imgs, test_labels)  
  accuracys_test.append(corrects / ( corrects + wrongs))
  costs_test.append(model.compute_cost(model.predict(test_imgs), test_labels_one_hot))


plt.plot(costs_train, c='b', label='cost train-set')
plt.plot(costs_test, c='r', label='cost test-set')
plt.xlabel("Interation")
plt.ylabel("Cost")
plt.legend()
plt.show()